In [1]:
!pip install -qU transformers torch sentencepiece accelerate>=0.20.1

In [2]:
import torch
from transformers import AutoTokenizer
import numpy as np

In [3]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
print(tokenizer.max_len_single_sentence)

510


In [4]:
text="Personally, I thought today was fine. Talking about APIs again really solidified my understanding of what they were. As for our group project, I don't feel great about it. Some of my team members have been collecting data for about two days straight. I can understand that some of our data sets can be harder to collect than others, but I feel like there's not tons of effort being put in by some of my team members. Today we decided to implement a new kind of data set to improve our model (recycling symbols on objects) and from what I can see on our Roboflow, only one team member collected and labeled his recycling symbol data. I don't know if my other team members just don't understand the idea, or if they are just not doing it. I'm planning on working on all of those other recycling labels tonight. "
tokens = tokenizer(text,
                   truncation=True,
                   padding="max_length",
                   max_length=200, return_tensors="pt")

In [5]:
tokens

{'input_ids': tensor([[    0, 40026,     6,    38,   802,   452,    21,  2051,     4, 14920,
            59, 35329,   456,   269,  2705,  3786,   127,  2969,     9,    99,
            51,    58,     4,   287,    13,    84,   333,   695,     6,    38,
           218,    75,   619,   372,    59,    24,     4,   993,     9,   127,
           165,   453,    33,    57,  8664,   414,    13,    59,    80,   360,
          1359,     4,    38,    64,  1346,    14,   103,     9,    84,   414,
          3880,    64,    28,  4851,     7,  5555,    87,   643,     6,    53,
            38,   619,   101,    89,    18,    45,  7741,     9,  1351,   145,
           342,    11,    30,   103,     9,   127,   165,   453,     4,  2477,
            52,  1276,     7,  5731,    10,    92,   761,     9,   414,   278,
             7,  1477,    84,  1421,    36, 43099, 21050, 19830,    15,  8720,
            43,     8,    31,    99,    38,    64,   192,    15,    84,  3005,
          1116,  5481,     6,   129,  

In [6]:
#To-Do: Use batch decode to generate Text sentence from tokens
ans = tokenizer.batch_decode(tokens['input_ids'], attention_mask=tokens['attention_mask'])
' '.join(ans)

"<s>Personally, I thought today was fine. Talking about APIs again really solidified my understanding of what they were. As for our group project, I don't feel great about it. Some of my team members have been collecting data for about two days straight. I can understand that some of our data sets can be harder to collect than others, but I feel like there's not tons of effort being put in by some of my team members. Today we decided to implement a new kind of data set to improve our model (recycling symbols on objects) and from what I can see on our Roboflow, only one team member collected and labeled his recycling symbol data. I don't know if my other team members just don't understand the idea, or if they are just not doing it. I'm planning on working on all of those other recycling labels tonight. </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>"

# Tokenize Twitter dataset

In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataframe = pd.read_csv('/content/drive/MyDrive/TweetData/new_dataset.csv')
dataframe.dropna()
dataframe.head()

,content,sentiment
0,damn first ye frank zappa,0
1,fantast fullon weekend go finish high cheeki c...,0
2,tri stay awak anyon tip keep awak,0
3,beat bos castelvania,0
4,problem think great thing reflect,0


In [ ]:
max_len = dataframe.shape[0] #Row
max_len

39996

In [ ]:
#To-Do: Find length of longest string of dataframe['content']
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
max_string_len = 0
for row in range(max_len):
  row_text = dataframe.iloc[row,0]
  words = word_tokenize(str(row_text))
  if len(words) > max_string_len:
    max_string_len = len(words)

In [ ]:
max_string_len

25

In [ ]:
shuffled_df = dataframe.sample(frac=1, random_state=42)

In [ ]:
input_ids = []
attention_masks = []
y_label = []
for i in range(0,1000):
  text = shuffled_df.iloc[i,0]
  label = shuffled_df.iloc[i,1]
  tokens = tokenizer(str(text), truncation=True, padding="max_length", max_length=max_string_len+5, return_tensors="pt")
  input_ids.append(tokens['input_ids'][0])
  attention_masks.append(tokens['attention_mask'][0])
  y_label.append(label)
  print(tokens)

{'input_ids': tensor([[  101,  4012, 18780, 18138, 15453,  8223, 29066,  7037,  4563, 13005,
         13184, 22747, 12722,  7513,  2072,  2215,  2663,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]])}
{'input_ids': tensor([[ 101, 2561, 4276, 2307, 9587, 5737, 4658, 3221, 3335, 8038, 3335, 3335,
         2047, 2259,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]])}
{'input_ids': tensor([[  101,  2019, 14573,  5684,  4183,  2334,  2173,  6805,  6497,  9530,
          7842, 12821,  9642,  2080,  2173,  2175, 25933,  2480,  9642,  3520,
          7712,  2232,  2485,   102,     0,     0,     0,     0,     0,     0]]), 'attention

In [ ]:
from torch.utils.data import Dataset

class TweetDataset(Dataset):
    def __init__(self, input_ids, attention_masks, y_labels):
        super().__init__()
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.y_labels = y_labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, index):
      ans = {"input_ids": self.input_ids[index],
                "attention_mask": self.attention_masks[index],
                "labels": self.y_labels[index]}
      return ans


In [ ]:
tweet_dataset = TweetDataset(input_ids, attention_masks, y_label)

In [ ]:
tweet_dataset[100]

{'input_ids': tensor([  101, 17111,  6517, 11338, 12458,  8569, 14141,  2072,  2228,  2298,
          2066,  2071,  5507, 19646, 10098,  2066, 22715,  9956,  7929, 22715,
          9956,   102,     0,     0,     0,     0,     0,     0,     0,     0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
         0, 0, 0, 0, 0, 0]),
 'labels': 4}

In [ ]:
import torch
path = '/content/drive/MyDrive/TweetData/my_dataset_1000.pt'
torch.save(tweet_dataset, path)